# Introduction: Manual Feature Engineering (part two)

In this notebook we will expand on the [Introduction to Manual Feature Engineering](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering/output) notebook. We will use the aggregation and value counting functions developed in that notebook in order to incorporate information from the `previous_application`, `POS_CASH_balance`, `installments_payments`, and `credit_card_balance` data files. We already used the information from the `bureau` and `bureau_balance` in the previous notebook and were able to improve our competition score compared to using only the `application` data. After running a model with the features included here, performance does increase, but we run into issues with an explosion in the number of features! I'm working on a notebook of feature selection, but for this notebook we will continue building up a rich set of data for our model. 

The definitions of the four additional data files are:

* previous_application (called `previous`): previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV.
* POS_CASH_BALANCE (called `cash`): monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.
* credit_card_balance (called `credit`): monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.
* installments_payment (called `installments`): payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment.

# Functions 

We spent quite a bit of time developing two functions in the previous notebook:

* `agg_numeric`: calculate aggregation statistics (`mean`, `count`, `max`, `min`) for numeric variables.
* `count_categorical`: compute counts and normalized counts of each category in a categorical variable.

Together, these two functions can extract information about both the numeric and categorical data in a dataframe. Our general approach will be to apply both of these functions to the dataframes, grouping by the client id, `SK_ID_CURR`. For the `POS_CASH_balance`, `credit_card_balance`, and `installment_payments`, we can first group by the `SK_ID_PREV`, the unique id for the previous loan. Then we will group the resulting dataframe by the `SK_ID_CURR` to calculate the aggregation statistics for each client across all of their previous loans. If that's a little confusing, I'd suggest heading back to the [first feature engineering notebook](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering/output).**

In [1]:
## Function to Aggregate Numeric Data

This groups by # pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

# Memory management
import gc 

## Function to Aggregate Numeric Data

This groups data by the `group_var` and calculates `mean`, `max`, `min`, and `sum`. It will only be applied to numeric data by default in pandas.

In [2]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    
    # First calculate counts
    counts = pd.DataFrame(df.groupby(group_var, as_index = False)[df.columns[1]].count()).rename(columns = {df.columns[1]: '%s_counts' % df_name})
    
    # Group by the specified variable and calculate the statistics
    agg = df.groupby(group_var).agg(['mean', 'max', 'min', 'sum']).reset_index()
    
    # Need to create new column names
    columns = [group_var]
    
    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
              
    #  Rename the columns
    agg.columns = columns
    
    # Merge with the counts
    agg = agg.merge(counts, on = group_var, how = 'left')
    
    return agg

### Function to Calculate Categorical Counts

This function calculates the occurrences (counts) of each category in a categorical variable for each client. It also calculates the normed count, which is the count for a category divided by the total counts for all categories in a categorical variable. 

In [3]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

### Function for KDE Plots of Variable

We also made a function that plots the distribution of variable colored by the value of `TARGET` (either 1 for did not repay the loan or 0 for did repay the loan). We can use this function to visually examine any new variables we create. This also calculates the correlation cofficient of the variable with the target which can be used as an approximation of whether or not the created variable will be useful. 

In [4]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)
    

Let's deal with one dataframe at a time. First up is the `previous_applications`. This has one row for every previous loan a client had at Home Credit. A client can have multiple previous loans which is why we need to aggregate statistics for each client.

### `previous_application`

In [5]:
previous = pd.read_csv('../input/previous_application.csv')
previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Calculate aggregate statistics for each numeric column
previous_agg = agg_numeric(previous.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR', df_name = 'previous_loans')
previous_agg.head()

,SK_ID_CURR,previous_loans_AMT_ANNUITY_mean,previous_loans_AMT_ANNUITY_max,previous_loans_AMT_ANNUITY_min,previous_loans_AMT_ANNUITY_sum,previous_loans_AMT_APPLICATION_mean,previous_loans_AMT_APPLICATION_max,previous_loans_AMT_APPLICATION_min,previous_loans_AMT_APPLICATION_sum,previous_loans_AMT_CREDIT_mean,previous_loans_AMT_CREDIT_max,previous_loans_AMT_CREDIT_min,previous_loans_AMT_CREDIT_sum,previous_loans_AMT_DOWN_PAYMENT_mean,previous_loans_AMT_DOWN_PAYMENT_max,previous_loans_AMT_DOWN_PAYMENT_min,previous_loans_AMT_DOWN_PAYMENT_sum,previous_loans_AMT_GOODS_PRICE_mean,previous_loans_AMT_GOODS_PRICE_max,previous_loans_AMT_GOODS_PRICE_min,previous_loans_AMT_GOODS_PRICE_sum,previous_loans_HOUR_APPR_PROCESS_START_mean,previous_loans_HOUR_APPR_PROCESS_START_max,previous_loans_HOUR_APPR_PROCESS_START_min,previous_loans_HOUR_APPR_PROCESS_START_sum,previous_loans_NFLAG_LAST_APPL_IN_DAY_mean,previous_loans_NFLAG_LAST_APPL_IN_DAY_max,previous_loans_NFLAG_LAST_APPL_IN_DAY_min,previous_loans_NFLAG_LAST_APPL_IN_DAY_sum,previous_loans_RATE_DOWN_PAYMENT_mean,previous_loans_RATE_DOWN_PAYMENT_max,previous_loans_RATE_DOWN_PAYMENT_min,previous_loans_RATE_DOWN_PAYMENT_sum,previous_loans_RATE_INTEREST_PRIMARY_mean,previous_loans_RATE_INTEREST_PRIMARY_max,previous_loans_RATE_INTEREST_PRIMARY_min,previous_loans_RATE_INTEREST_PRIMARY_sum,previous_loans_RATE_INTEREST_PRIVILEGED_mean,previous_loans_RATE_INTEREST_PRIVILEGED_max,previous_loans_RATE_INTEREST_PRIVILEGED_min,previous_loans_RATE_INTEREST_PRIVILEGED_sum,previous_loans_DAYS_DECISION_mean,previous_loans_DAYS_DECISION_max,previous_loans_DAYS_DECISION_min,previous_loans_DAYS_DECISION_sum,previous_loans_SELLERPLACE_AREA_mean,previous_loans_SELLERPLACE_AREA_max,previous_loans_SELLERPLACE_AREA_min,previous_loans_SELLERPLACE_AREA_sum,previous_loans_CNT_PAYMENT_mean,previous_loans_CNT_PAYMENT_max,previous_loans_CNT_PAYMENT_min,previous_loans_CNT_PAYMENT_sum,previous_loans_DAYS_FIRST_DRAWING_mean,previous_loans_DAYS_FIRST_DRAWING_max,previous_loans_DAYS_FIRST_DRAWING_min,previous_loans_DAYS_FIRST_DRAWING_sum,previous_loans_DAYS_FIRST_DUE_mean,previous_loans_DAYS_FIRST_DUE_max,previous_loans_DAYS_FIRST_DUE_min,previous_loans_DAYS_FIRST_DUE_sum,previous_loans_DAYS_LAST_DUE_1ST_VERSION_mean,previous_loans_DAYS_LAST_DUE_1ST_VERSION_max,previous_loans_DAYS_LAST_DUE_1ST_VERSION_min,previous_loans_DAYS_LAST_DUE_1ST_VERSION_sum,previous_loans_DAYS_LAST_DUE_mean,previous_loans_DAYS_LAST_DUE_max,previous_loans_DAYS_LAST_DUE_min,previous_loans_DAYS_LAST_DUE_sum,previous_loans_DAYS_TERMINATION_mean,previous_loans_DAYS_TERMINATION_max,previous_loans_DAYS_TERMINATION_min,previous_loans_DAYS_TERMINATION_sum,previous_loans_NFLAG_INSURED_ON_APPROVAL_mean,previous_loans_NFLAG_INSURED_ON_APPROVAL_max,previous_loans_NFLAG_INSURED_ON_APPROVAL_min,previous_loans_NFLAG_INSURED_ON_APPROVAL_sum,previous_loans_counts
0,100001,3951.000,3951.000,3951.000,3951.000,24835.50,24835.5,24835.5,24835.5,23787.00,23787.0,23787.0,23787.0,2520.0,2520.0,2520.0,2520.0,24835.5,24835.5,24835.5,24835.5,13.000000,13,13,13,1.0,1,1,1,0.104326,0.104326,0.104326,0.104326,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,-1740.0,-1740,-1740,-1740,23.0,23,23,23,8.0,8.0,8.0,8.0,365243.0,365243.0,365243.0,365243.0,-1709.000000,-1709.0,-1709.0,-1709.0,-1499.000000,-1499.0,-1499.0,-1499.0,-1619.000000,-1619.0,-1619.0,-1619.0,-1612.000000,-1612.0,-1612.0,-1612.0,0.000000,0.0,0.0,0.0,1
1,100002,9251.775,9251.775,9251.775,9251.775,179055.00,179055.0,179055.0,179055.0,179055.00,179055.0,179055.0,179055.0,0.0,0.0,0.0,0.0,179055.0,179055.0,179055.0,179055.0,9.000000,9,9,9,1.0,1,1,1,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,-606.0,-606,-606,-606,500.0,500,500,500,24.0,24.0,24.0,24.0,365243.0,365243.0,365243.0,365243.0,-565.000000,-565.0,-565.0,-565.0,125.000000,125.0,125.0,125.0,-25.000000,-25.0,-25.0,-25.0,-17.000000,-17.0,-17.0,-17.0,0.000000,0.0,0.0,0.0,1
2,100003,56553.990,98356.995,6737.310,169661.970,435436.50,900000.0,68809.5,1306309.5,484191.

In [7]:
# Calculate value counts for each categorical column
previous_counts = count_categorical(previous, group_var = 'SK_ID_CURR', df_name = 'previous_loans')
previous_counts.head()

,previous_loans_NAME_CONTRACT_TYPE_Cash loans_count,previous_loans_NAME_CONTRACT_TYPE_Cash loans_count_norm,previous_loans_NAME_CONTRACT_TYPE_Consumer loans_count,previous_loans_NAME_CONTRACT_TYPE_Consumer loans_count_norm,previous_loans_NAME_CONTRACT_TYPE_Revolving loans_count,previous_loans_NAME_CONTRACT_TYPE_Revolving loans_count_norm,previous_loans_NAME_CONTRACT_TYPE_XNA_count,previous_loans_NAME_CONTRACT_TYPE_XNA_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_FRIDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_FRIDAY_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_MONDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_MONDAY_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_SATURDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_SATURDAY_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_SUNDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_SUNDAY_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_THURSDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_THURSDAY_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_TUESDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_TUESDAY_count_norm,previous_loans_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_count,previous_loans_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_count_norm,previous_loans_FLAG_LAST_APPL_PER_CONTRACT_N_count,previous_loans_FLAG_LAST_APPL_PER_CONTRACT_N_count_norm,previous_loans_FLAG_LAST_APPL_PER_CONTRACT_Y_count,previous_loans_FLAG_LAST_APPL_PER_CONTRACT_Y_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Building a house or an annex_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Building a house or an annex_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Business development_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Business development_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a garage_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a garage_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a home_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a home_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a new car_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a new car_count_norm,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a used car_count,previous_loans_NAME_CASH_LOAN_PURPOSE_Buying a used car_count_norm,...,previous_loans_NAME_YIELD_GROUP_low_action_count,previous_loans_NAME_YIELD_GROUP_low_action_count_norm,previous_loans_NAME_YIELD_GROUP_low_normal_count,previous_loans_NAME_YIELD_GROUP_low_normal_count_norm,previous_loans_NAME_YIELD_GROUP_middle_count,previous_loans_NAME_YIELD_GROUP_middle_count_norm,previous_loans_PRODUCT_COMBINATION_Card Street_count,previous_loans_PRODUCT_COMBINATION_Card Street_count_norm,previous_loans_PRODUCT_COMBINATION_Card X-Sell_count,previous_loans_PRODUCT_COMBINATION_Card X-Sell_count_norm,previous_loans_PRODUCT_COMBINATION_Cash_count,previous_loans_PRODUCT_COMBINATION_Cash_count_norm,previous_loans_PRODUCT_COMBINATION_Cash Street: high_count,previous_loans_PRODUCT_COMBINATION_Cash Street: high_count_norm,previous_loans_PRODUCT_COMBINATION_Cash Street: low_count,previous_loans_PRODUCT_COMBINATION_Cash Street: low_count_norm,previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count,previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm,previous_loans_PRODUCT_COMBINATION_Cash X-Sell: high_count,previous_loans_PRODUCT_COMBINATION_Cash X-Sell: high_count_norm,previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count,previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm,previous_loans_PRODUCT_COMBINATION_Cash X-Sell: middle_count,previous_loans_PRODUCT_COMBINATION_Cash X-Sell: middle_count_norm,previous_loans_PRODUCT_COMBINATION_POS household with interest_count,previous_loans_PRODUCT_COMBINATION_POS household with interest_count_norm,previous_loans_PRODUCT_COMBINATION_POS household without interest_count,previous_loa

In [8]:
print('Previous aggregated shape: ', previous_agg.shape)
print('Previous categorical counts shape: ', previous_counts.shape)

Previous aggregated shape:  (338857, 78)
Previous categorical counts shape:  (338857, 286)


We can join the calculated dataframe to the main training dataframe using a merge. Then we should delete the calculated dataframes to avoid using too much of the kernel memory.

In [9]:
train = pd.read_csv('../input/application_train.csv')
test = pd.read_csv('../input/application_test.csv')

# Merge in the previous information
train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

test = test.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
test = test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

# Remove variables to free memory
gc.enable()
del previous, previous_agg, previous_counts
gc.collect()

56

We are going to have to be careful about calculating too many features. We don't want to overwhelm the model with too many irrelevant features or features with too many missing values. In the previous notebook, we removed any features with more than 75% missing values. To be consistent, we will apply that same logic here. 

## Function to Calculate Missing Values

In [10]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df, print_info = False):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        if print_info:
            # Print some summary information
            print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
                "There are " + str(mis_val_table_ren_columns.shape[0]) +
                  " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [11]:
def remove_missing_columns(train, test, threshold = 90):
    # Calculate missing stats for train and test (remember to calculate a percent!)
    train_miss = pd.DataFrame(train.isnull().sum())
    train_miss['percent'] = 100 * train_miss[0] / len(train)
    
    test_miss = pd.DataFrame(test.isnull().sum())
    test_miss['percent'] = 100 * test_miss[0] / len(test)
    
    # list of missing columns for train and test
    missing_train_columns = list(train_miss.index[train_miss['percent'] > threshold])
    missing_test_columns = list(test_miss.index[test_miss['percent'] > threshold])
    
    # Combine the two lists together
    missing_columns = list(set(missing_train_columns + missing_test_columns))
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_columns), threshold))
    
    # Drop the missing columns and return
    train = train.drop(columns = missing_columns)
    test = test.drop(columns = missing_columns)
    
    return train, test

In [13]:
train, test = remove_missing_columns(train, test)

There are 6 columns with greater than 90% missing values.


# Applying to More Data

### Function to Aggregate Stats at the Client Level

In [14]:
def aggregate_client(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level 
    at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan 
        and then the client (example ['SK_ID_PREV', 'SK_ID_CURR'])
        names (list of two strings): names to call the resulting columns
        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
        Each client will have a single row with all the numeric data aggregated
    """
    
    # Aggregate the numeric columns
    df_agg = agg_numeric(df, group_var = group_vars[0], df_name = df_names[0])
    
    # If there are categorical variables
    if any(df.dtypes == 'object'):
    
        # Count the categorical columns
        df_counts = count_categorical(df, group_var = group_vars[0], df_name = df_names[0])

        # Merge the numeric and categorical
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Merge to get the client id in dataframe
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, group_var = group_vars[1], df_name = df_names[1])

        
    # No categorical variables
    else:
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, group_var = group_vars[1], df_name = df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

## Monthly Cash Data

In [15]:
cash = pd.read_csv('../input/POS_CASH_balance.csv')
cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [16]:
cash_by_client = aggregate_client(cash, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
cash_by_client.head()

,SK_ID_CURR,client_cash_NAME_CONTRACT_STATUS_Active_count_mean,client_cash_NAME_CONTRACT_STATUS_Active_count_max,client_cash_NAME_CONTRACT_STATUS_Active_count_min,client_cash_NAME_CONTRACT_STATUS_Active_count_sum,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_mean,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_min,client_cash_NAME_CONTRACT_STATUS_Active_count_norm_sum,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_mean,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_max,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_min,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_sum,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_mean,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_max,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_min,client_cash_NAME_CONTRACT_STATUS_Amortized debt_count_norm_sum,client_cash_NAME_CONTRACT_STATUS_Approved_count_mean,client_cash_NAME_CONTRACT_STATUS_Approved_count_max,client_cash_NAME_CONTRACT_STATUS_Approved_count_min,client_cash_NAME_CONTRACT_STATUS_Approved_count_sum,client_cash_NAME_CONTRACT_STATUS_Approved_count_norm_mean,client_cash_NAME_CONTRACT_STATUS_Approved_count_norm_max,client_cash_NAME_CONTRACT_STATUS_Approved_count_norm_min,client_cash_NAME_CONTRACT_STATUS_Approved_count_norm_sum,client_cash_NAME_CONTRACT_STATUS_Canceled_count_mean,client_cash_NAME_CONTRACT_STATUS_Canceled_count_max,client_cash_NAME_CONTRACT_STATUS_Canceled_count_min,client_cash_NAME_CONTRACT_STATUS_Canceled_count_sum,client_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_mean,client_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_max,client_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_min,client_cash_NAME_CONTRACT_STATUS_Canceled_count_norm_sum,client_cash_NAME_CONTRACT_STATUS_Completed_count_mean,client_cash_NAME_CONTRACT_STATUS_Completed_count_max,client_cash_NAME_CONTRACT_STATUS_Completed_count_min,client_cash_NAME_CONTRACT_STATUS_Completed_count_sum,client_cash_NAME_CONTRACT_STATUS_Completed_count_norm_mean,client_cash_NAME_CONTRACT_STATUS_Completed_count_norm_max,client_cash_NAME_CONTRACT_STATUS_Completed_count_norm_min,...,client_cash_CNT_INSTALMENT_FUTURE_sum_max,client_cash_CNT_INSTALMENT_FUTURE_sum_min,client_cash_CNT_INSTALMENT_FUTURE_sum_sum,client_cash_SK_DPD_mean_mean,client_cash_SK_DPD_mean_max,client_cash_SK_DPD_mean_min,client_cash_SK_DPD_mean_sum,client_cash_SK_DPD_max_mean,client_cash_SK_DPD_max_max,client_cash_SK_DPD_max_min,client_cash_SK_DPD_max_sum,client_cash_SK_DPD_min_mean,client_cash_SK_DPD_min_max,client_cash_SK_DPD_min_min,client_cash_SK_DPD_min_sum,client_cash_SK_DPD_sum_mean,client_cash_SK_DPD_sum_max,client_cash_SK_DPD_sum_min,client_cash_SK_DPD_sum_sum,client_cash_SK_DPD_DEF_mean_mean,client_cash_SK_DPD_DEF_mean_max,client_cash_SK_DPD_DEF_mean_min,client_cash_SK_DPD_DEF_mean_sum,client_cash_SK_DPD_DEF_max_mean,client_cash_SK_DPD_DEF_max_max,client_cash_SK_DPD_DEF_max_min,client_cash_SK_DPD_DEF_max_sum,client_cash_SK_DPD_DEF_min_mean,client_cash_SK_DPD_DEF_min_max,client_cash_SK_DPD_DEF_min_min,client_cash_SK_DPD_DEF_min_sum,client_cash_SK_DPD_DEF_sum_mean,client_cash_SK_DPD_DEF_sum_max,client_cash_SK_DPD_DEF_sum_min,client_cash_SK_DPD_DEF_sum_sum,client_cash_counts_mean,client_cash_counts_max,client_cash_counts_min,client_cash_counts_sum,client_counts
0,100001,3.555556,4,3,32.0,0.777778,0.800000,0.750000,7.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,1.000000,1,1,9.0,0.222222,0.250000,0.200000,...,10.0,3.0,62.0,0.777778,1.75,0.0,7.0,3.111111,7,0,28,0.0,0,0,0,3.111111,7,0,28,0.777778,1.75,0.0,7.0,3.111111,7,0,28,0.0,0,0,0,3.111111,7,0,28,4.555556,5,4,41,9
1,100002,19.000000,19,19,361.0,1.000000,1.000000,1.000000,19.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.000000,0,0,0.0,0.000000,0.000000,0.000000,...,285.0,285.0,5415.0,0.000000,0.00,0.0,0.0,0.000000,0,0,0,0.0,0,0,0,0.000000,0,0,0,0.000000,0.00

In [17]:
print('Cash by Client Shape: ', cash_by_client.shape)
train = train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del cash, cash_by_client
gc.collect()

Cash by Client Shape:  (337252, 174)


35

In [18]:
train, test = remove_missing_columns(train, test)

There are 0 columns with greater than 90% missing values.


## Monthly Credit Data

In [ ]:
credit = pd.read_csv('../input/credit_card_balance.csv')
credit.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [ ]:
credit_by_client = aggregate_client(credit, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
credit_by_client.head()

In [ ]:
print('Credit by client shape: ', credit_by_client.shape)

train = train.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del credit, credit_by_client
gc.collect()

In [ ]:
train, test = remove_missing_columns(train, test)

### Installment Payments

In [ ]:
installments = pd.read_csv('../input/installments_payments.csv')
installments.head()

In [ ]:
installments_by_client = aggregate_client(installments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
installments_by_client.head()

In [ ]:
print('Installments by client shape: ', installments_by_client.shape)

train = train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del installments, installments_by_client
gc.collect()

In [ ]:
train, test = remove_missing_columns(train, test)

In [ ]:
print('Final Training Shape: ', train.shape)
print('Final Testing Shape: ', test.shape)

 #### Save All Newly Calculated Features
 
 Unfortunately, saving all the created features does not work in a Kaggle notebook. You will have to run the code on your personal machine. I have run the code and uploaded the [entire datasets here](https://www.kaggle.com/willkoehrsen/home-credit-manual-engineered-features). I plan on doing some feature selection and uploading reduced versions of the datasets. Right now, they are slightly to big to handle in Kaggle notebooks or scripts. .

In [ ]:
# train.to_csv('train_previous_raw.csv', index = False, chunksize = 500)
# test.to_csv('test_previous_raw.csv', index = False)